# 1. ウィキペディアから必要な情報をスクレイピングしよう

### 下準備
 - 必要なライブラリのインストール
 - conda install -c anaconda requests
 - conda install -c anaconda beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
response_obj = requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text
soup = BeautifulSoup(response_obj)
Wards_Tokyo_Table = soup.find('table', {'class':'wikitable sortable'})

### 必要なデータを取得する
 - 2:区の名前
 - 4:人口
 - 5:密度（/km2）
 - 6:区の面積(km2)
 - 7:主な都市名
 - 必要な情報の中にいらないタグや改行コードが含まれているのでそれを除去する必要がある

In [3]:
name = []
pop = []
dence = []
area = []
majar_city = []

for row in Wards_Tokyo_Table.findAll("tr"):
    Ward = row.findAll('td')
    # ヘッダー行は飛ばす
    if len(Ward) == 8:
            # 区の名前を取得する
            v1 = Ward[2]
            name.append(v1.getText())
            
            # 区の人口を取得する
            v2 = Ward[4]
            # tdタグの中に不要なspanタグが入っていたので、消す
            [v2.extract() for v2 in soup('span')]
            pop.append(v2.getText())
            
            # 区の人口密度を取得する
            v3 = Ward[5]
            dence.append(v3.getText())
        
            # 区の面積を取得する
            v4 = Ward[6]
            # 右側の改行は必要ないので削除
            area.append(v4.getText().rstrip())
            
            v5 = Ward[7]
            # 改行を削除し、カンマで分割、最初の名称だけを取得する
            majar_city.append(v5.getText().rstrip().split(',')[0])

print(name)
print(pop)
print(dence)
print(area)
print(majar_city)

['Chiyoda', 'Chūō', 'Minato', 'Shinjuku', 'Bunkyō', 'Taitō', 'Sumida', 'Kōtō', 'Shinagawa', 'Meguro', 'Ōta', 'Setagaya', 'Shibuya', 'Nakano', 'Suginami', 'Toshima', 'Kita', 'Arakawa', 'Itabashi', 'Nerima', 'Adachi', 'Katsushika', 'Edogawa']
['59,441', '147,620', '248,071', '339,211', '223,389', '200,486', '260,358', '502,579', '392,492', '280,283', '722,608', '910,868', '227,850', '332,902', '570,483', '294,673', '345,063', '213,648', '569,225', '726,748', '674,067', '447,140', '685,899']
['5,100', '14,460', '12,180', '18,620', '19,790', '19,830', '18,910', '12,510', '17,180', '19,110', '11,910', '15,690', '15,080', '21,350', '16,750', '22,650', '16,740', '21,030', '17,670', '15,120', '12,660', '12,850', '13,750']
['11.66', '10.21', '20.37', '18.22', '11.29', '10.11', '13.77', '40.16', '22.84', '14.67', '60.66', '58.05', '15.11', '15.59', '34.06', '13.01', '20.61', '10.16', '32.22', '48.08', '53.25', '34.80', '49.90']
['Nagatachō', 'Nihonbashi', 'Odaiba', 'Shinjuku', 'Hongō', 'Ueno', '

### 取得したデータをpandasに変換する

In [4]:
import pandas as pd

data = []
data.append(name)
data.append(pop)
data.append(dence)
data.append(area)
data.append(majar_city)

# リストからpandasへ
df = pd.DataFrame(data,index=["name","population","dence","area","major_district"]) 
# 列と行を反転
df = df.T
df

,name,population,dence,area,major_district
0,Chiyoda,"59,441","5,100",11.66,Nagatachō
1,Chūō,"147,620","14,460",10.21,Nihonbashi
2,Minato,"248,071","12,180",20.37,Odaiba
3,Shinjuku,"339,211","18,620",18.22,Shinjuku
4,Bunkyō,"223,389","19,790",11.29,Hongō
5,Taitō,"200,486","19,830",10.11,Ueno
6,Sumida,"260,358","18,910",13.77,Kinshichō
7,Kōtō,"502,579","12,510",40.16,Kiba
8,Shinagawa,"392,492","17,180",22.84,Shinagawa
9,Meguro,"280,283","19,110",14.67,Meguro


# 2. Geopyを使って位置情報を取得してみよう

### geopy
 - "conda install -c conda-forge geopy"でインストール
 - バージョンは1.19.0
 - user_agentは無いとエラーが出るが、何でも良い

In [5]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="Tokyo")

location = geolocator.geocode("Nihonbashi")
print(location.latitude, location.longitude)

location = geolocator.geocode("Odaiba")
print(location.latitude, location.longitude)


35.684058 139.774501377979
35.61912805 139.779403349221


### 地区の名称を元に経度と井戸を取得してみる
 - KibaとKasaiで変な値が帰ってきてるが気にしないことにする,kasaiがコンゴ民主共和国になってしまった
 - 毎回サーバに問い合わせるので実行には数秒時間がかかるよ
 - 経度と緯度をセットでリストに保存します

In [6]:
lat = []
lon = []
for n in majar_city:
    location = geolocator.geocode(n)
    lat.append(location.latitude)
    lon.append(location.longitude)

### 位置情報をpandasのデータフレームに追加する
 - カラム名はlocation

In [7]:
df["latitude"] = lat
df["longitude"] = lon

In [8]:
df.to_csv("tokyo.csv")

In [9]:
df

,name,population,dence,area,major_district,latitude,longitude
0,Chiyoda,"59,441","5,100",11.66,Nagatachō,35.675618,139.743469
1,Chūō,"147,620","14,460",10.21,Nihonbashi,35.684058,139.774501
2,Minato,"248,071","12,180",20.37,Odaiba,35.619128,139.779403
3,Shinjuku,"339,211","18,620",18.22,Shinjuku,35.693763,139.703632
4,Bunkyō,"223,389","19,790",11.29,Hongō,35.175386,137.013430
5,Taitō,"200,486","19,830",10.11,Ueno,35.711788,139.776096
6,Sumida,"260,358","18,910",13.77,Kinshichō,35.696752,139.814151
7,Kōtō,"502,579","12,510",40.16,Kiba,23.013134,-80.832875
8,Shinagawa,"392,492","17,180",22.84,Shinagawa,35.599252,139.738910
9,Meguro,"280,283","19,110",14.67,Meguro,35.621250,139.688014
